Determines the best model for each value of the hyperparameters in `for_each`, by looking at their predicted MAE in an evaluation file.  
**Should not be done, confounds validation and test**

In [3]:
main_dir = r'/home/janik.fechtelpeter/Documents/bptt/results/MRT1_Transformer_Valid_Smoothed_Imputed'
eval_dir = '00_summary_7stepsahead'
epoch_criterion = 'latest'
mode = 'min'
for_each = ['participant', 'train_on_data_until_timestep']
include_hyper = {}
exclude_hyper = {'feature': ['EMA_emotion_control', 'EMA_emotion_change']}
save_to = '/home/janik.fechtelpeter/Documents/bptt/results/MRT1_Transformer_Valid_Smoothed_Imputed_Best'


import sys
sys.path.append('..')
import os
from typing import Iterable
from tqdm import tqdm
import pandas as pd
import eval_reallabor_utils
import shutil

eval_dir = os.path.join(main_dir, eval_dir)
results = pd.read_csv(os.path.join(eval_dir, 'evaluation.csv'))

results = eval_reallabor_utils.include_exclude_hypers(results, include_hyper, exclude_hyper)
results['abs_diff'] = (results['ground_truth'] - results['prediction']).abs()
features = results['feature'].unique()
if len(for_each) > 0:
    grouped = results.groupby(for_each)
    print('Extracting best model for ' + ", ".join([f"{for_each[k]}" for k in range(len(for_each))]))
else:
    grouped = [('all', results)]
    print('Extracting best model over all.')
pbar = tqdm(grouped, total=grouped.ngroups)
for name, group in tqdm(grouped, total=grouped.ngroups):
    if len(for_each)>0:
        if not isinstance(name, Iterable):
            name = (name, )
    if mode == 'min':
        model_id, run = group.groupby(['model_id', 'run'])['abs_diff'].sum().sort_values(ascending=True).index[0]
    elif mode == 'max':
        model_id, run = group.groupby(['model_id', 'run'])['abs_diff'].sum().sort_values(ascending=False).index[0]
    run = str(run).zfill(3)
    model_dir = os.path.join(main_dir, model_id, run)
    if len(grouped)>1:
        shutil.copytree(model_dir, os.path.join(save_to, "_".join([f"{for_each[k]}_{name[k]}" for k in range(len(for_each))])))
    else:
        shutil.copytree(model_dir, save_to)

Extracting best model for participant==12
Extracting best model for participant==13
Extracting best model for participant==15
Extracting best model for participant==17
Extracting best model for participant==28
Extracting best model for participant==34
Extracting best model for participant==35
Extracting best model for participant==38
Extracting best model for participant==52
Extracting best model for participant==54
Extracting best model for participant==57
Extracting best model for participant==61
Extracting best model for participant==62


/tmp/ipykernel_4177160/336003973.py:31: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in grouped:
